In [1]:
import pandas as pd
import arff as liacarff

# 加载 ARFF 文件
with open('PROMISE_DOSSPRE_OLD_3.arff', 'r') as f:
    data_dict = liacarff.load(f)

# 提取数据
data = data_dict['data']

# 转换为 DataFrame
df = pd.DataFrame(data, columns=[attr[0] for attr in data_dict['attributes']])
df['ProjectID'] = df['ProjectID'].astype(int)  # 将 ProjectID 列转换为整数类型

# 显示 DataFrame
print(df)

      ProjectID                                    RequirementText _class_
0             1  The system shall refresh the display every 60 ...      PE
1             1  The application shall match the color of the s...      LF
2             1  If projected  the data must be readable.  On a...      US
3             1  The product shall be available during normal b...       A
4             1  If projected  the data must be understandable....      US
...         ...                                                ...     ...
2052          9  The database may trade off fidelity through ca...      FT
2053          9  The API shall have master topology replicating...      FT
2054          9  The system must parse, filter, transform and s...      FT
2055          9  The application shall employ real-user monitor...      FT
2056          9  The software should apply graceful degradation...      FT

[2057 rows x 3 columns]


In [2]:
# DATA PREPROCESSING
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import string

# 下载NLTK的停用词和词性标注器数据
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# # lower
# df['RequirementText'] = df['RequirementText'].str.lower()

# # # Remove punctuation, leading and trailing spaces
# df['RequirementText'] = df['RequirementText'].apply(lambda x: re.sub(f"[{re.escape(string.punctuation)}]", "", x.strip()))

# # 分词
df['RequirementText'] = df['RequirementText'].apply(word_tokenize)

# # stopwords
# stop_words = set(stopwords.words('english'))
# df['RequirementText'] = df['RequirementText'].apply(lambda x: [word for word in x if word not in stop_words])

# # stemming
stemmer = PorterStemmer()
df['RequirementText'] = df['RequirementText'].apply(lambda x: [stemmer.stem(word) for word in x])

# # lemmatization
# lemmatizer = WordNetLemmatizer()
# df['RequirementText'] = df['RequirementText'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# # 合并词语为字符串
df['RequirementText'] = df['RequirementText'].apply(lambda x: ' '.join(x))

# Replace multiple spaces with a single space
df['RequirementText'] = df['RequirementText'].apply(lambda x: re.sub(' +', ' ', x))

# 移除整个列中每个字符串首尾的全部空格
df['RequirementText'] = df['RequirementText'].str.strip()

# 查看处理后的数据集
print(df.head())

[nltk_data] Downloading package stopwords to /home/li/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/li/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/li/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


   ProjectID                                    RequirementText _class_
0          1  the system shall refresh the display everi 60 ...      PE
1          1  the applic shall match the color of the schema...      LF
2          1  if project the data must be readabl . on a 10x...      US
3          1  the product shall be avail dure normal busi ho...       A
4          1  if project the data must be understand . on a ...      US


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [4]:
from sklearn.model_selection import train_test_split
y = df['_class_']
X = df['RequirementText']

# 假设 X 是特征，y 是目标变量（类别标签）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

print(y_test)
class_distribution = y_test.value_counts()
print(class_distribution)

1071    US
954     PE
1104    PE
1612     L
1338    PE
        ..
1263    PE
1505    SE
383     LF
1454    US
579     LF
Name: _class_, Length: 206, dtype: object
_class_
F     44
PE    25
SE    24
US    22
MN    16
LF    16
O     13
A     12
SC    10
L      9
PO     8
FT     7
Name: count, dtype: int64


In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 初始化SVM模型
svm_model = SVC(kernel='linear', C=1.0)

# 训练模型
svm_model.fit(X_train_tfidf, y_train)

# 在测试集上进行预测
y_pred_svm = svm_model.predict(X_test_tfidf)

# 评估模型性能
print("Accuracy (SVM):", accuracy_score(y_test, y_pred_svm))
print("Classification Report (SVM):\n", classification_report(y_test, y_pred_svm))

Accuracy (SVM): 0.6796116504854369
Classification Report (SVM):
               precision    recall  f1-score   support

           A       0.69      0.75      0.72        12
           F       0.67      0.91      0.77        44
          FT       0.50      0.14      0.22         7
           L       0.50      0.33      0.40         9
          LF       0.82      0.56      0.67        16
          MN       0.44      0.44      0.44        16
           O       0.75      0.69      0.72        13
          PE       0.57      0.64      0.60        25
          PO       1.00      0.62      0.77         8
          SC       0.89      0.80      0.84        10
          SE       0.76      0.67      0.71        24
          US       0.74      0.77      0.76        22

    accuracy                           0.68       206
   macro avg       0.69      0.61      0.63       206
weighted avg       0.69      0.68      0.67       206



In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 定义SVM模型
svm_model = SVC()

# 定义要调优的超参数网格
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}

# 创建GridSearchCV对象
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='f1_weighted')

# 执行Grid Search以选择最佳超参数配置
grid_search.fit(X_train_tfidf, y_train)

# 输出最佳超参数配置
print("Best hyperparameters: ", grid_search.best_params_)

# 使用最佳超参数配置的模型进行预测
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_tfidf)

# 输出在测试集上的准确率
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test set: {:.2f}%".format(accuracy * 100))
print("Classification Report (SVM):\n", classification_report(y_test, y_pred))


Best hyperparameters:  {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy on test set: 73.30%
Classification Report (SVM):
               precision    recall  f1-score   support

           A       0.64      0.75      0.69        12
           F       0.73      1.00      0.85        44
          FT       0.50      0.14      0.22         7
           L       0.75      0.33      0.46         9
          LF       0.69      0.69      0.69        16
          MN       0.44      0.44      0.44        16
           O       0.83      0.77      0.80        13
          PE       0.68      0.68      0.68        25
          PO       0.86      0.75      0.80         8
          SC       0.89      0.80      0.84        10
          SE       0.81      0.71      0.76        24
          US       0.90      0.82      0.86        22

    accuracy                           0.73       206
   macro avg       0.73      0.66      0.67       206
weighted avg       0.73      0.73      0.72       206



In [7]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 定义欠采样器
under_sampler = RandomUnderSampler(random_state=42)

# 对训练集进行欠采样
X_train_resampled, y_train_resampled = under_sampler.fit_resample(X_train_tfidf, y_train)

# 在欠采样后的数据上训练模型（这里使用了SVM作为示例）
svm_model = SVC()
svm_model.fit(X_train_resampled, y_train_resampled)

# 对测试集进行转换
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 在测试集上进行预测
y_pred = svm_model.predict(X_test_tfidf)

# 评估模型性能
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6407766990291263
Classification Report:
               precision    recall  f1-score   support

           A       0.44      0.67      0.53        12
           F       0.79      0.68      0.73        44
          FT       0.25      0.14      0.18         7
           L       0.30      0.33      0.32         9
          LF       0.82      0.56      0.67        16
          MN       0.53      0.50      0.52        16
           O       0.75      0.69      0.72        13
          PE       0.48      0.60      0.54        25
          PO       0.62      1.00      0.76         8
          SC       0.89      0.80      0.84        10
          SE       0.62      0.67      0.64        24
          US       0.89      0.77      0.83        22

    accuracy                           0.64       206
   macro avg       0.62      0.62      0.61       206
weighted avg       0.66      0.64      0.64       206



In [8]:
# F binary

# df['binary_label'] = df['_class_'].apply(lambda x: 1 if x == 'F' else 0)
# X_train, X_test, y_train, y_test = train_test_split(df['RequirementText'], df['binary_label'], test_size=0.2, random_state=42)

# # 使用TfidfVectorizer进行转换
# ngram_range = (1,2)
# min_df = 0.01 # ignore terms that appear in less than 1% of the documents
# max_df = 0.8 # ignore terms that appear in more than 80% of the documents

# tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
#                         ngram_range=ngram_range,
#                         stop_words=None,
#                         lowercase=False,
#                         max_df=max_df,
#                         min_df=min_df,
#                         norm='l2',
#                         sublinear_tf=True)

# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)

# model_tfidf = MultinomialNB()
# model_tfidf.fit(X_train_tfidf, y_train)

# y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

# print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
# print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf))

In [9]:
# filtered_df = df[df['_class_'] != 'F']
# X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# # 使用TfidfVectorizer进行转换
# ngram_range = (1,2)
# min_df = 0.01 # ignore terms that appear in less than 1% of the documents
# max_df = 0.8 # ignore terms that appear in more than 80% of the documents

# tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
#                         ngram_range=ngram_range,
#                         stop_words=None,
#                         lowercase=False,
#                         max_df=max_df,
#                         min_df=min_df,
#                         norm='l2',
#                         sublinear_tf=True)

# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)

# model_tfidf = MultinomialNB()
# model_tfidf.fit(X_train_tfidf, y_train)

# y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

# print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
# print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

In [10]:
filtered_df = df[df['_class_'].isin(['MN', 'SC', 'PE'])]
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

Accuracy (TF-IDF): 0.8725490196078431
Classification Report (TF-IDF):
               precision    recall  f1-score   support

          MN       0.90      0.79      0.84        33
          PE       0.83      0.98      0.90        50
          SC       1.00      0.74      0.85        19

    accuracy                           0.87       102
   macro avg       0.91      0.83      0.86       102
weighted avg       0.88      0.87      0.87       102



In [11]:
filtered_df = df[df['_class_'].isin(['A', 'FT'])]
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

Accuracy (TF-IDF): 0.8461538461538461
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           A       0.82      0.96      0.88        24
          FT       0.91      0.67      0.77        15

    accuracy                           0.85        39
   macro avg       0.87      0.81      0.83        39
weighted avg       0.86      0.85      0.84        39



In [12]:
filtered_df = df[df['_class_'].isin(['SE', 'L'])]
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

Accuracy (TF-IDF): 0.8307692307692308
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           L       1.00      0.35      0.52        17
          SE       0.81      1.00      0.90        48

    accuracy                           0.83        65
   macro avg       0.91      0.68      0.71        65
weighted avg       0.86      0.83      0.80        65



In [13]:
filtered_df = df[df['_class_'].isin(['O', 'US'])]
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

Accuracy (TF-IDF): 0.9571428571428572
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           O       0.96      0.92      0.94        26
          US       0.96      0.98      0.97        44

    accuracy                           0.96        70
   macro avg       0.96      0.95      0.95        70
weighted avg       0.96      0.96      0.96        70

